In [4]:
root_dir = "/data_x/guro/DATA/ASCO_RAW"
save_dir = "/data_x/guro/DATA/ASCO_RAW/CUSTOM_CHUNKS"

In [5]:
import pandas as pd
raw_corpus_path = '/data_x/guro/DATA/ASCO_RAW/raw_corpus.parquet'
df_raw = pd.read_parquet(raw_corpus_path)

In [12]:
df_raw

,doc_id,contents,metadata
0,Denosumab Prevents Bone Loss and Microarchitec...,Estradiol plays an important role in breast ca...,"{'doc_authors': 'Sabashini K. Ramchand, MBBS, ..."
1,Denosumab Prevents Bone Loss and Microarchitec...,"This was a 12-month randomized, double-blind, ...","{'doc_authors': 'Sabashini K. Ramchand, MBBS, ..."
2,Denosumab Prevents Bone Loss and Microarchitec...,Participants were stratified by lumbar spine B...,"{'doc_authors': 'Sabashini K. Ramchand, MBBS, ..."
3,Denosumab Prevents Bone Loss and Microarchitec...,Participants received 60 mg DMAB or matching P...,"{'doc_authors': 'Sabashini K. Ramchand, MBBS, ..."
4,Denosumab Prevents Bone Loss and Microarchitec...,Adverse events were documented at each visit a...,"{'doc_authors': 'Sabashini K. Ramchand, MBBS, ..."
...,...,...,...
7440,Cost-Effectiveness Analysis of No Adjuvant The...,The population modeled was similar to the popu...,"{'doc_authors': 'Matthew C. Ward, MD https://o..."
7441,Cost-Effectiveness Analysis of No Adjuvant The...,Table 2presents the results from the base case...,"{'doc_authors': 'Matthew C. Ward, MD https://o..."
7442,Cost-Effectiveness Analysis of No Adjuvant The...,One-way sensitivity analyses were performed to...,"{'doc_authors': 'Matthew C. Ward, MD https://o..."
7443,Cost-Effectiveness Analysis of No Adjuvant The...,Probabilistic sensitivity analysis using Monte...,"{'doc_authors': 'Matthew C. Ward, MD https://o..."


---
# 1. 문장 단위

In [1]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /mnt/raid6/yjoonjang/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [2]:
# NLTK 설치
# pip install nltk

from nltk.tokenize import sent_tokenize


# 텍스트 파일 읽기
text = """
Python is an interpreted, high-level, general-purpose programming language.
Its design philosophy emphasizes code readability.
It is widely used in various domains such as web development, data analysis, and AI.
"""

# 문장을 토큰화
sentences = sent_tokenize(text)

# 특정 크기로 청킹 (예: 2문장씩)
chunk_size = 1
chunks = [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]

# 결과 출력
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {' '.join(chunk)}")

Chunk 1: 
Python is an interpreted, high-level, general-purpose programming language.
Chunk 2: Its design philosophy emphasizes code readability.
Chunk 3: It is widely used in various domains such as web development, data analysis, and AI.


In [14]:
chunked_data = []

for index, data in enumerate(df_raw["contents"]):
    sentences = sent_tokenize(data)
    chunks = [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]
    for i, chunk in enumerate(chunks):
        chunked_data.append({
            "contents": " ".join(chunk),
            "metadata": df_raw["metadata"][index],
            "doc_id": df_raw["doc_id"][index]
        })


chunked_df = pd.DataFrame(chunked_data)
chunked_df.to_parquet(save_dir + "/sentence_chunks.parquet")

In [15]:
# 저장한 데이터 보기
loaded_chunked_df = pd.read_parquet(save_dir + "/sentence_chunks.parquet")
print(loaded_chunked_df.head())

                                            contents  \
0  Estradiol plays an important role in breast ca...   
1  Adjuvant endocrine therapies prevent breast ca...   
2  In premenopausal women with ER-positive early-...   
3  This was a 12-month randomized, double-blind, ...   
4  Premenopausal women, between age 18 and 55 yea...   

                                            metadata  \
0  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...   
1  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...   
2  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...   
3  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...   
4  {'doc_authors': 'Sabashini K. Ramchand, MBBS, ...   

                                              doc_id  
0  Denosumab Prevents Bone Loss and Microarchitec...  
1  Denosumab Prevents Bone Loss and Microarchitec...  
2  Denosumab Prevents Bone Loss and Microarchitec...  
3  Denosumab Prevents Bone Loss and Microarchitec...  
4  Denosumab Prevents Bone Loss and Microarchitec..

# 2. 단어 단위

In [16]:
# 1000_200
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter_1000_200 = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

text_splitter_500_100 = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)

text_splitter_300_50 = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [19]:
chunked_data_1000 = []
chunked_data_500 = []
chunked_data_300 = []

for index, data in enumerate(df_raw["contents"]):
    chunks_1000 = text_splitter_1000_200.split_text(data)
    chunks_500 = text_splitter_500_100.split_text(data)
    chunks_300 = text_splitter_300_50.split_text(data)
    for chunk in chunks_1000:
        chunked_data_1000.append({
            "contents": chunk,
            "metadata": df_raw["metadata"][index],
            "doc_id": df_raw["doc_id"][index]
        })
    for chunk in chunks_500:
        chunked_data_500.append({
            "contents": chunk,
            "metadata": df_raw["metadata"][index],
            "doc_id": df_raw["doc_id"][index]
        })
    for chunk in chunks_300:
        chunked_data_300.append({
            "contents": chunk,
            "metadata": df_raw["metadata"][index],
            "doc_id": df_raw["doc_id"][index]
        })


In [20]:
chunked_df_1000 = pd.DataFrame(chunked_data_1000)
chunked_df_500 = pd.DataFrame(chunked_data_500)
chunked_df_300 = pd.DataFrame(chunked_data_300)

chunked_df_1000.to_parquet(save_dir + "/word_chunks_1000_200.parquet")
chunked_df_500.to_parquet(save_dir + "/word_chunks_500_100.parquet")
chunked_df_300.to_parquet(save_dir + "/word_chunks_300_50.parquet")